In [17]:
%pip install groq

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from groq import Groq

In [19]:
os.environ['GROQ_API_KEY'] = "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"

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [20]:
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

True

In [21]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if len(self.messages) > 10:
            self.messages = [self.messages[0]] + self.messages[-9:]
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        try:
            completion = self.client.chat.completions.create(
                model="llama3-70b-8192",
                messages=self.messages
            )
            if not completion.choices:
                raise ValueError("Resposta vazia da API")
            return completion.choices[0].message.content
        except Exception as e:
            print(f"Erro na API: {e}")
            return f"Erro: {e}"
    
    def reset(self):
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})

In [22]:
def class_words(frase):
    tokens = word_tokenize(frase)
    tags = pos_tag(tokens)
    return str(tags)

In [23]:
def classifie_sentence(sentence):
    system_prompt = """
    You run in a loop of Thought, Action, PAUSE, Observation.
    At the end of the loop you output an Answer.
    Use Thought to describe your thoughts about the question you have been asked.
    Use Action to run one of the actions available to you - then return PAUSE.
    Observation will be the result of running those actions.

    Your available actions are:
    class_words:
    e.g. class_words(frase)
    returns a list with the words and their grammatical class

    Example session:
    Frase: The black cat jumped over the tall brick wall.
    Thought: I need to analyze the grammatical structure of this sentence.
    Action: class_words(The black cat jumped over the tall brick wall)
    PAUSE

    Observation: [('The', 'DT'), ('black', 'JJ'), ('cat', 'NN'), ('jumped', 'VBD'), ('over', 'IN'), ('the', 'DT'), ('tall', 'JJ'), ('brick', 'NN'), ('wall', 'NN')]

    Answer: Here are the grammatical classifications:
    [('The', 'DT'), ('black', 'JJ'), ('cat', 'NN'), ('jumped', 'VBD'), ('over', 'IN'), ('the', 'DT'), ('tall', 'JJ'), ('brick', 'NN'), ('wall', 'NN')]
    """.strip()

    agent = Agent(client, system_prompt)
    response = agent(f"Analyze this sentence: {sentence}")
    agent.reset()
    
    while True:
        if "PAUSE" in response:
            action_line = next((line for line in response.split('\n') if line.startswith("Action:")), None)
            if action_line:
                action = action_line.split("Action:")[1].strip()
                if action.startswith("class_words("):
                    frase = action[len("class_words("):-1].strip()
                    if not frase: 
                        frase = sentence
                    observation = class_words(frase)
                    response = agent(f"Observation: {observation}")
                else:
                    response = agent("Observation: Unknown action")
            else:
                response = agent("Observation: No action specified")
        
        elif "Answer:" in response:
            return response.split("Answer:")[1].strip()
        
        else:
            response = agent()

In [24]:
if __name__ == "__main__":
    input_sentence = "The black cat jumps over the tall brick wall, falling on a dog."
    result = classifie_sentence(input_sentence)
    print(f"Analysis result for '{input_sentence}':")
    print(result)

Analysis result for 'The black cat jumps over the tall brick wall, falling on a dog.':
Here are the grammatical classifications: [('The', 'DT'), ('black', 'JJ'), ('cat', 'NN'), ('jumps', 'NNS'), ('over', 'IN'), ('the', 'DT'), ('tall', 'JJ'), ('brick', 'NN'), ('wall', 'NN'), (',', ','), ('falling', 'VBG'), ('on', 'IN'), ('a', 'DT'), ('dog', 'NN')]
